In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, StringType
import json
import ast

In [0]:
df_semantic_classified = spark.table("gold.conversations_semantic_classified")

In [0]:
with open('/Workspace/Users/ammarbagharib@gmail.com/categories_list.json', 'r') as f:
    categories_list = json.load(f)
print(categories_list)

In [0]:
# Apply ai_classify
df_classified = (
    df_semantic_classified
    .filter(F.col("classification_method") != "semantic")
    .select(
        "conversation_id",
        "ISID",
        "country",
        "combined_text",
        "convo_summary",
        "char_count",
        F.expr(f"""
            ai_classify(
                combined_text,
                array({','.join([f'"{c}"' for c in categories_list])})
            )
        """).alias("top_category"),
        F.lit("llm").alias("classification_method")
    )
)

# Get semantic classified rows
df_semantic = (
    df_semantic_classified
    .filter(F.col("classification_method") == "semantic")
    .select(
        "conversation_id",
        "ISID",
        "country",
        "combined_text",
        "convo_summary",
        "char_count",
        F.col("top_category"),
        "classification_method"
    )
)

# Union both DataFrames
df_classified_all = df_classified.unionByName(df_semantic)

# Write to table
spark.sql("DROP TABLE gold.conversations_classified")
df_classified_all.write.mode("overwrite").saveAsTable("gold.conversations_classified")

In [0]:
# display(df_classified)